In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sympy import dsolve, symbols, Function, Eq
import sympy
import polars as pl
import torch
import gif
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from PIL import Image
import math

In [ ]:
SAVE_GIFS = True

## free particle - static reference frame

$$ L = T - V = \frac{1}{2} m \left( \dot{x}^2 + \dot{y}^2 \right) $$

$$ \frac{dL}{dx} = \frac{d}{dt} \frac{dL}{d\dot{x}} $$

$$ \frac{dL}{dy} = \frac{d}{dt} \frac{dL}{d\dot{y}} $$

$$ 0 = m \ddot{x} $$

$$ 0 = m \ddot{y} $$

In [ ]:
x = symbols("x", cls=Function)
t, m = symbols("t m")
x(t)

In [ ]:
diffeq = Eq(m * x(t).diff(t, t), 0)
diffeq

In [ ]:
dsolve(diffeq)

$$ x = C_1 + C_2 t $$

$$ y = C_3 + C_4 t $$

In [ ]:
def calc_next_position(t: float, position: float, velocity: float) -> float:
    return position + velocity * t


calc_next_position(1, 0, 1)

In [ ]:
def calc_next_positions(
    t_values: torch.Tensor, position: float, velocity: float
) -> list[float]:
    return [calc_next_position(float(t), position, velocity) for t in t_values]


# Initial condition
x0, vx0 = 0, -1
y0, vy0 = 1, 1

t_values = torch.linspace(0, 2 * math.pi, 201)

x_positions = calc_next_positions(t_values, x0, vx0)
y_positions = calc_next_positions(t_values, y0, vy0)

trajectory = pl.DataFrame({"t": t_values.tolist(), "x": x_positions, "y": y_positions})
trajectory.head(2)

In [ ]:
def animate_trajectory(trajectory: pl.DataFrame, title: str, filename: str):
    x_vals = trajectory["x"].to_list()
    y_vals = trajectory["y"].to_list()

    x_min = min(x_vals) - 1
    x_max = max(x_vals) + 1

    y_min = min(y_vals) - 1
    y_max = max(y_vals) + 1

    frames = []

    @gif.frame
    def plot_frame(i: int) -> Figure:
        fig, ax = plt.subplots()

        ax.plot(x_vals[: i + 1], y_vals[: i + 1], color="lightblue", linewidth=2)
        ax.plot(x_vals[i], y_vals[i], "bo", markersize=8)

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

        ax.set_xlabel("x")
        ax.set_ylabel("y")

        ax.set_title(title)

        return fig

    if SAVE_GIFS:
        for i in range(len(x_vals)):
            frames.append(plot_frame(i))

        gif.save(frames, filename, duration=300)


animate_trajectory(
    trajectory,
    "Animated Particle Trajectory in Static Reference Frame",
    "trajectory.gif",
)

## free particle - constantly rotating reference frame

$$ [x, y] \cdot R = [C_1 + C_2 t, C_3 + C_4 t] \cdot R $$

where $R$ is the rotation matrix whose values depend on $\theta(t)$, see [wiki](https://en.wikipedia.org/wiki/Rotating_reference_frame).

In [ ]:
def calc_rotating_reference_frame(
    t: float, x: float, y: float, freq: float, theta0: float
) -> tuple[float, float]:
    theta = theta0 + t * freq
    c = math.cos(-theta)
    s = math.sin(-theta)
    x_rot = x * c - y * s
    y_rot = x * s + y * c
    return x_rot, y_rot


calc_rotating_reference_frame(1.0, x_positions[0], y_positions[0], 0.1, 0.0)

In [ ]:
theta0 = 0.0
freq = 1.0


def calc_rotated_positions(
    t_values: torch.Tensor,
    x_positions: list[float],
    y_positions: list[float],
    freq: float,
    theta0: float,
) -> tuple[list[float], list[float]]:
    xy_rots = [
        calc_rotating_reference_frame(float(t), x, y, freq, theta0)
        for t, x, y in zip(t_values, x_positions, y_positions, strict=True)
    ]
    x_rots = [v[0] for v in xy_rots]
    y_rots = [v[1] for v in xy_rots]
    return x_rots, y_rots


x_positions_rot, y_positions_rot = calc_rotated_positions(
    t_values, x_positions, y_positions, freq, theta0
)

trajectory_rotating = pl.DataFrame(
    {"t": t_values.tolist(), "x": x_positions_rot, "y": y_positions_rot}
)

In [ ]:
animate_trajectory(
    trajectory_rotating,
    "Animated Particle Trajectory in Rotating Reference Frame",
    "trajectory-rotating.gif",
)